# 📘 Phase 2: Macro Divergence and Trade Dynamics Analysis


This notebook explores macroeconomic relationships between:
- **GDP divergence (Nominal vs. Real)**
- **Net export growth or change**
- **Other macroeconomic indicators** such as fiscal balance, inflation, consumption, savings, and interest rates

We will evaluate correlations, temporal relationships, and structural shifts across historical regimes.


## 🔧 Step 1: Setup and FRED API Access

In [7]:

from fredapi import Fred
import pandas as pd

# Initialize FRED connection (insert your key)
fred = Fred(api_key="306a3b46d80923127981cdabb3c9691f")


## 📦 Step 2: Download Core Macroeconomic Series

In [8]:
# Step 2: Define and download macroeconomic series

# Define FRED series
series_ids = {
    "GDPCA": "Nominal GDP",
    "GDPC1": "Real GDP",
    "GNPCA": "Gross National Product",
    "EXPGSCA": "Exports of Goods and Services",
    "IMPGSCA": "Imports of Goods and Services",
    "FYONGDA188S": "Federal Outlays",
    "FYFRGDA188S": "Federal Receipts",
    "FEDFUNDS": "Effective Federal Funds Rate",
    "GS10": "10-Year Treasury Constant Maturity Rate",
    "CPIAUCSL": "Consumer Price Index (CPI-U)",
    "PCEPI": "PCE Price Index",
    "PSAVERT": "Personal Savings Rate",
    "PCEC": "Nominal Personal Consumption Expenditures",
    "PCECC96": "Real Personal Consumption Expenditures"
}

# Create dictionary to store time series
data = {}

# Download and inspect
for series_id, label in series_ids.items():
    print(f"🔹 Downloading: {label} ({series_id})")
    series = fred.get_series(series_id)
    df = series.reset_index()
    df.columns = ['Date', series_id]
    df['Date'] = pd.to_datetime(df['Date'])
    data[series_id] = df

    # Inspection
    print("Preview:")
    print(df.head(2))
    print(f"Type: {df[series_id].dtype} | Missing: {df[series_id].isna().sum()} values")
    print("-" * 60)

print("✅ All series downloaded and inspected.")

🔹 Downloading: Nominal GDP (GDPCA)
Preview:
        Date     GDPCA
0 1929-01-01  1191.124
1 1930-01-01  1089.785
Type: float64 | Missing: 0 values
------------------------------------------------------------
🔹 Downloading: Real GDP (GDPC1)
Preview:
        Date     GDPC1
0 1947-01-01  2182.681
1 1947-04-01  2176.892
Type: float64 | Missing: 0 values
------------------------------------------------------------
🔹 Downloading: Gross National Product (GNPCA)
Preview:
        Date     GNPCA
0 1929-01-01  1202.659
1 1930-01-01  1100.670
Type: float64 | Missing: 0 values
------------------------------------------------------------
🔹 Downloading: Exports of Goods and Services (EXPGSCA)
Preview:
        Date  EXPGSCA
0 1929-01-01   41.725
1 1930-01-01   34.486
Type: float64 | Missing: 0 values
------------------------------------------------------------
🔹 Downloading: Imports of Goods and Services (IMPGSCA)
Preview:
        Date  IMPGSCA
0 1929-01-01   51.857
1 1930-01-01   45.108
Type: float64

In [9]:
import os

# Output path (update if needed)
output_path = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\fred_series_master.xlsx"

# Save all series into one Excel file with tabs
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for series_id, df in data.items():
        # Limit sheet names to 31 characters
        sheet_name = series_id if len(series_id) <= 31 else series_id[:28] + "..."
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ All series saved to: {output_path}")


✅ All series saved to: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\fred_series_master.xlsx


In [10]:
from openpyxl import load_workbook

# Load the workbook
wb = load_workbook(r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\fred_series_master.xlsx")

# List all sheet names
print("✅ Sheet tabs found:")
print(wb.sheetnames)

# Preview first few values from one sheet (e.g., GDPCA)
ws = wb['GDPCA']
for row in ws.iter_rows(min_row=1, max_row=5, values_only=True):
    print(row)


✅ Sheet tabs found:
['GDPCA', 'GDPC1', 'GNPCA', 'EXPGSCA', 'IMPGSCA', 'FYONGDA188S', 'FYFRGDA188S', 'FEDFUNDS', 'GS10', 'CPIAUCSL', 'PCEPI', 'PSAVERT', 'PCEC', 'PCECC96']
('Date', 'GDPCA')
(datetime.datetime(1929, 1, 1, 0, 0), 1191.124)
(datetime.datetime(1930, 1, 1, 0, 0), 1089.785)
(datetime.datetime(1931, 1, 1, 0, 0), 1019.977)
(datetime.datetime(1932, 1, 1, 0, 0), 888.414)


## 🔁 Step 3: Preprocess, Merge and Align All Series

In [11]:
# ✅ Use this only if data['FEDFUNDS'], etc. are missing or not loaded
from fredapi import Fred
import pandas as pd

fred = Fred(api_key="306a3b46d80923127981cdabb3c9691f")

monthly_series = ["FEDFUNDS", "GS10", "CPIAUCSL", "PCEPI", "PSAVERT"]
resampled_quarterly = {}

for series_id in monthly_series:
    print(f"🔄 Resampling {series_id} to quarterly average...")
    df = fred.get_series(series_id).reset_index()
    df.columns = ['Date', series_id]
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df_q = df.resample('Q').mean().reset_index()
    resampled_quarterly[series_id] = df_q

# Display the resampled FEDFUNDS table using pandas (top 10 rows)
print("📊 Resampled FEDFUNDS (Quarterly Average):")
print(resampled_quarterly["FEDFUNDS"].head(10))
print("✅ Resampling complete. Quarterly averages calculated."
)

🔄 Resampling FEDFUNDS to quarterly average...


C:\Users\troyr\AppData\Local\Temp\ipykernel_12180\3847424248.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_q = df.resample('Q').mean().reset_index()


🔄 Resampling GS10 to quarterly average...


C:\Users\troyr\AppData\Local\Temp\ipykernel_12180\3847424248.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_q = df.resample('Q').mean().reset_index()


🔄 Resampling CPIAUCSL to quarterly average...


C:\Users\troyr\AppData\Local\Temp\ipykernel_12180\3847424248.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_q = df.resample('Q').mean().reset_index()


🔄 Resampling PCEPI to quarterly average...


C:\Users\troyr\AppData\Local\Temp\ipykernel_12180\3847424248.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_q = df.resample('Q').mean().reset_index()


🔄 Resampling PSAVERT to quarterly average...
📊 Resampled FEDFUNDS (Quarterly Average):
        Date  FEDFUNDS
0 1954-09-30  1.030000
1 1954-12-31  0.986667
2 1955-03-31  1.343333
3 1955-06-30  1.500000
4 1955-09-30  1.940000
5 1955-12-31  2.356667
6 1956-03-31  2.483333
7 1956-06-30  2.693333
8 1956-09-30  2.810000
9 1956-12-31  2.926667
✅ Resampling complete. Quarterly averages calculated.


C:\Users\troyr\AppData\Local\Temp\ipykernel_12180\3847424248.py:16: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df_q = df.resample('Q').mean().reset_index()


In [12]:
import os
import pandas as pd

# Folder to save individual CSVs and Excel file
output_dir = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data"
excel_path = os.path.join(output_dir, "resampled_quarterly_macro.xlsx")

# Save each series to individual CSVs
for series_id, df in resampled_quarterly.items():
    csv_path = os.path.join(output_dir, f"{series_id}_quarterly.csv")
    df.to_csv(csv_path, index=False)
    print(f"✅ Saved CSV: {csv_path}")

# Save all to one Excel file (multi-sheet)
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    for series_id, df in resampled_quarterly.items():
        sheet_name = series_id if len(series_id) <= 31 else series_id[:28] + "..."
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✅ All resampled series saved to Excel: {excel_path}")


✅ Saved CSV: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\FEDFUNDS_quarterly.csv
✅ Saved CSV: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\GS10_quarterly.csv
✅ Saved CSV: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\CPIAUCSL_quarterly.csv
✅ Saved CSV: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\PCEPI_quarterly.csv
✅ Saved CSV: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\PSAVERT_quarterly.csv

✅ All resampled series saved to Excel: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\resampled_quarterly_macro.xlsx


In [1]:
from fredapi import Fred
import pandas as pd
import os

# Initialize FRED connection
fred = Fred(api_key="306a3b46d80923127981cdabb3c9691f")

# Output folder
output_dir = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data"

# Define new demand- and supply-side series
new_series = {
    "UNRATE": "Unemployment Rate (U-3)",
    "CIVPART": "Labor Force Participation Rate",
    "PCEC": "Nominal Personal Consumption Expenditures",
    "DSPIC96": "Real Disposable Personal Income",
    "INDPRO": "Industrial Production Index",
    "IPMAN": "Manufacturing Production Index",
    "HOUST": "Housing Starts (Total)",
    "PERMIT": "Building Permits",
    "TCU": "Capacity Utilization (Total Industry)"
}

# Pull and save each to an individual CSV
for series_id, label in new_series.items():
    print(f"📥 Pulling {label} ({series_id})...")
    series = fred.get_series(series_id).reset_index()
    series.columns = ['Date', series_id]
    series['Date'] = pd.to_datetime(series['Date'])

    # Save to CSV
    output_path = os.path.join(output_dir, f"{series_id}_raw.csv")
    series.to_csv(output_path, index=False)
    print(f"✅ Saved to {output_path}\n")


📥 Pulling Unemployment Rate (U-3) (UNRATE)...
✅ Saved to C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\UNRATE_raw.csv

📥 Pulling Labor Force Participation Rate (CIVPART)...
✅ Saved to C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\CIVPART_raw.csv

📥 Pulling Nominal Personal Consumption Expenditures (PCEC)...
✅ Saved to C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\PCEC_raw.csv

📥 Pulling Real Disposable Personal Income (DSPIC96)...
✅ Saved to C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\DSPIC96_raw.csv

📥 Pulling Industrial Production Index (INDPRO)...
✅ Saved to C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\INDPRO_raw.csv

📥 Pulling Manufacturing Production Index (IPMAN)...
✅ Saved to C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\I

In [2]:
import pandas as pd
import os

# Folder path where raw CSVs are stored
data_dir = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data"

# List of FRED series IDs to resample
series_ids = [
    "UNRATE",
    "CIVPART",
    "PCEC",
    "DSPIC96",
    "INDPRO",
    "IPMAN",
    "HOUST",
    "PERMIT",
    "TCU"
]

# Dictionary to store resampled DataFrames
quarterly_data = {}

# Resample each series
for series_id in series_ids:
    file_path = os.path.join(data_dir, f"{series_id}_raw.csv")
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Resample to quarterly average
    df_q = df.resample('Q-DEC').mean().reset_index()
    quarterly_data[series_id] = df_q

    print(f"✅ Resampled {series_id} to quarterly")

# Save to Excel with one sheet per series
output_excel = os.path.join(data_dir, "macro_additional_quarterly.xlsx")
with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    for series_id, df_q in quarterly_data.items():
        sheet_name = series_id if len(series_id) <= 31 else series_id[:28] + "..."
        df_q.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n📁 All resampled series saved to Excel:\n{output_excel}")


C:\Users\troyr\AppData\Local\Temp\ipykernel_11648\3198427289.py:31: FutureWarning: 'Q-DEC' is deprecated and will be removed in a future version, please use 'QE-DEC' instead.
  df_q = df.resample('Q-DEC').mean().reset_index()
C:\Users\troyr\AppData\Local\Temp\ipykernel_11648\3198427289.py:31: FutureWarning: 'Q-DEC' is deprecated and will be removed in a future version, please use 'QE-DEC' instead.
  df_q = df.resample('Q-DEC').mean().reset_index()
C:\Users\troyr\AppData\Local\Temp\ipykernel_11648\3198427289.py:31: FutureWarning: 'Q-DEC' is deprecated and will be removed in a future version, please use 'QE-DEC' instead.
  df_q = df.resample('Q-DEC').mean().reset_index()
C:\Users\troyr\AppData\Local\Temp\ipykernel_11648\3198427289.py:31: FutureWarning: 'Q-DEC' is deprecated and will be removed in a future version, please use 'QE-DEC' instead.
  df_q = df.resample('Q-DEC').mean().reset_index()
C:\Users\troyr\AppData\Local\Temp\ipykernel_11648\3198427289.py:31: FutureWarning: 'Q-DEC' is de

✅ Resampled UNRATE to quarterly
✅ Resampled CIVPART to quarterly
✅ Resampled PCEC to quarterly
✅ Resampled DSPIC96 to quarterly
✅ Resampled INDPRO to quarterly
✅ Resampled IPMAN to quarterly
✅ Resampled HOUST to quarterly
✅ Resampled PERMIT to quarterly
✅ Resampled TCU to quarterly

📁 All resampled series saved to Excel:
C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\macro_additional_quarterly.xlsx


In [3]:
import pandas as pd
import os

# Define input files
data_dir = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data"
core_path = os.path.join(data_dir, "resampled_quarterly_macro.xlsx")
addon_path = os.path.join(data_dir, "macro_additional_quarterly.xlsx")

# Function to load all sheets into one DataFrame list
def load_excel_series(filepath):
    all_sheets = pd.read_excel(filepath, sheet_name=None)
    series_list = []
    for name, df in all_sheets.items():
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.rename(columns={df.columns[1]: name})
        series_list.append(df)
    return series_list

# Load both sets
core_series = load_excel_series(core_path)
addon_series = load_excel_series(addon_path)

# Merge all series on 'Date'
master_df = None
for df in core_series + addon_series:
    if master_df is None:
        master_df = df
    else:
        master_df = pd.merge(master_df, df, on='Date', how='outer')

# Sort and save
master_df = master_df.sort_values('Date').reset_index(drop=True)
output_path = os.path.join(data_dir, "master_macro_quarterly.xlsx")
master_df.to_excel(output_path, index=False)

# ✅ Inspect first few rows
print("\n📊 Preview of merged master macro DataFrame:")
print(master_df.head(10))

print(f"\n✅ Merged Excel file saved to:\n{output_path}")



📊 Preview of merged master macro DataFrame:
        Date  FEDFUNDS  GS10  CPIAUCSL  PCEPI  PSAVERT  UNRATE  CIVPART  PCEC  \
0 1919-03-31       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
1 1919-06-30       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
2 1919-09-30       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
3 1919-12-31       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
4 1920-03-31       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
5 1920-06-30       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
6 1920-09-30       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
7 1920-12-31       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
8 1921-03-31       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   
9 1921-06-30       NaN   NaN       NaN    NaN      NaN     NaN      NaN   NaN   

   DSPIC96    INDPRO  IPMAN  HOUST  PERMIT  TCU  
0      NaN  4

In [4]:
import pandas as pd
import os

# Define your input and output file paths
data_dir = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data"
input_file = os.path.join(data_dir, "master_macro_quarterly.xlsx")
output_file = os.path.join(data_dir, "master_macro_quarterly_trimmed.xlsx")

# Load the master macro dataset
master_df = pd.read_excel(input_file)
master_df['Date'] = pd.to_datetime(master_df['Date'])

# Add column to count missing values in each row
master_df['Missing_Count'] = master_df.isna().sum(axis=1)

# Filter to rows from 1950 onward
clean_df = master_df[master_df['Date'].dt.year >= 1950]

# Find the first row with no missing values
first_full_index = clean_df[clean_df['Missing_Count'] == 0].index.min()

# Trim the dataset from that row forward
trimmed_df = master_df.loc[first_full_index:].drop(columns=['Missing_Count']).reset_index(drop=True)

# Save the cleaned DataFrame to a new Excel file
trimmed_df.to_excel(output_file, index=False)

# Preview info
print(f"✅ Trimmed dataset starts at: {trimmed_df['Date'].iloc[0].date()}")
print(f"📁 Saved cleaned dataset to:\n{output_file}")
print("\n📊 Preview:")
print(trimmed_df.head(10))


✅ Trimmed dataset starts at: 1972-03-31
📁 Saved cleaned dataset to:
C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\master_macro_quarterly_trimmed.xlsx

📊 Preview:
        Date   FEDFUNDS      GS10   CPIAUCSL      PCEPI    PSAVERT    UNRATE  \
0 1972-03-31   3.546667  6.033333  41.333333  21.080333  12.366667  5.766667   
1 1972-06-30   4.300000  6.143333  41.600000  21.202000  11.633333  5.700000   
2 1972-09-30   4.743333  6.290000  41.933333  21.388333  11.966667  5.566667   
3 1972-12-31   5.146667  6.373333  42.366667  21.562667  13.433333  5.366667   
4 1973-03-31   6.536667  6.603333  43.033333  21.822667  12.533333  4.933333   
5 1973-06-30   7.816667  6.806667  43.933333  22.242000  13.333333  4.933333   
6 1973-09-30  10.560000  7.206667  44.800000  22.646667  13.400000  4.800000   
7 1973-12-31   9.996667  6.753333  45.933333  23.110667  14.533333  4.766667   
8 1974-03-31   9.323333  7.053333  47.300000  23.797667  13.966667  5.13333

In [5]:
import pandas as pd
import os

# File paths
data_dir = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data"
core_path = os.path.join(data_dir, "resampled_quarterly_macro.xlsx")
addon_path = os.path.join(data_dir, "macro_additional_quarterly.xlsx")
output_path = os.path.join(data_dir, "master_macro_quarterly_labeled.xlsx")

# Helper function to load and trim all series
def load_and_trim(filepath, start_date="1972-03-31"):
    sheets = pd.read_excel(filepath, sheet_name=None)
    cleaned = []
    for name, df in sheets.items():
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[df['Date'] >= pd.to_datetime(start_date)].reset_index(drop=True)
        df = df.rename(columns={df.columns[1]: name})
        cleaned.append(df)
    return cleaned

# Load and trim all series
core_series = load_and_trim(core_path)
addon_series = load_and_trim(addon_path)

# Merge all series on 'Date'
master_df = None
for df in core_series + addon_series:
    if master_df is None:
        master_df = df
    else:
        master_df = pd.merge(master_df, df, on='Date', how='outer')

# Rename columns with descriptive names
rename_dict = {
    "GDPCA": "Nominal GDP",
    "GDPC1": "Real GDP",
    "GNPCA": "Gross National Product",
    "EXPGSCA": "Exports",
    "IMPGSCA": "Imports",
    "FYONGDA188S": "Federal Outlays",
    "FYFRGDA188S": "Federal Receipts",
    "FEDFUNDS": "Fed Funds Rate",
    "GS10": "10-Year Treasury Yield",
    "CPIAUCSL": "CPI (All Urban Consumers)",
    "PCEPI": "PCE Price Index",
    "PSAVERT": "Personal Savings Rate",
    "PCEC": "Nominal Personal Consumption",
    "PCECC96": "Real Personal Consumption",
    "UNRATE": "Unemployment Rate",
    "CIVPART": "Labor Force Participation Rate",
    "DSPIC96": "Real Disposable Income",
    "INDPRO": "Industrial Production Index",
    "IPMAN": "Manufacturing Production Index",
    "HOUST": "Housing Starts",
    "PERMIT": "Building Permits",
    "TCU": "Capacity Utilization (Total Industry)"
}

master_df = master_df.rename(columns=rename_dict)

# Save to Excel
master_df.to_excel(output_path, index=False)

# Show preview
print(f"✅ Saved to: {output_path}")
print("📊 Preview:")
print(master_df.head(10))


✅ Saved to: C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\master_macro_quarterly_labeled.xlsx
📊 Preview:
        Date  Fed Funds Rate  10-Year Treasury Yield  \
0 1972-03-31        3.546667                6.033333   
1 1972-06-30        4.300000                6.143333   
2 1972-09-30        4.743333                6.290000   
3 1972-12-31        5.146667                6.373333   
4 1973-03-31        6.536667                6.603333   
5 1973-06-30        7.816667                6.806667   
6 1973-09-30       10.560000                7.206667   
7 1973-12-31        9.996667                6.753333   
8 1974-03-31        9.323333                7.053333   
9 1974-06-30       11.250000                7.543333   

   CPI (All Urban Consumers)  PCE Price Index  Personal Savings Rate  \
0                  41.333333        21.080333              12.366667   
1                  41.600000        21.202000              11.633333   
2                  

In [1]:
import pandas as pd
import os

# Define folder paths
base_dir = r"C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis"
quarterly_dir = os.path.join(base_dir, "data", "quarterly")
output_file = os.path.join(quarterly_dir, "master_macro_quarterly_complete_labeled.xlsx")

# Load trimmed base data
trimmed_path = os.path.join(quarterly_dir, "master_macro_quarterly_trimmed.xlsx")
master_df = pd.read_excel(trimmed_path)
master_df['Date'] = pd.to_datetime(master_df['Date'])

# Load all sheets from resampled_quarterly_macro.xlsx
resampled_path = os.path.join(quarterly_dir, "resampled_quarterly_macro.xlsx")
resampled_sheets = pd.read_excel(resampled_path, sheet_name=None)

# Merge in any additional series not already in the trimmed dataset
existing_cols = set(master_df.columns)

for series_id, df in resampled_sheets.items():
    if series_id in existing_cols:
        continue  # Already in dataset
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Date'] >= pd.to_datetime("1972-03-31")].reset_index(drop=True)
    master_df = pd.merge(master_df, df, on='Date', how='left')

# Rename to descriptive column names
rename_dict = {
    "GDPCA": "Nominal GDP",
    "GDPC1": "Real GDP",
    "GNPCA": "Gross National Product",
    "EXPGSCA": "Exports",
    "IMPGSCA": "Imports",
    "FYONGDA188S": "Federal Outlays",
    "FYFRGDA188S": "Federal Receipts",
    "FEDFUNDS": "Fed Funds Rate",
    "GS10": "10-Year Treasury Yield",
    "CPIAUCSL": "CPI (Urban Consumers)",
    "PCEPI": "PCE Price Index",
    "PSAVERT": "Personal Savings Rate",
    "PCEC": "Nominal Consumption",
    "PCECC96": "Real Consumption",
    "UNRATE": "Unemployment Rate",
    "CIVPART": "Labor Force Participation",
    "DSPIC96": "Real Disposable Income",
    "INDPRO": "Industrial Production",
    "IPMAN": "Manufacturing Output",
    "HOUST": "Housing Starts",
    "PERMIT": "Building Permits",
    "TCU": "Capacity Utilization"
}

master_df = master_df.rename(columns=rename_dict)

# Save final labeled output
master_df.to_excel(output_file, index=False)

# Preview
print(f"✅ Final master workbook saved to:\n{output_file}")
print("\n📊 Columns:")
print(master_df.columns.tolist())
print("\n📅 First date in dataset:", master_df['Date'].min().date())
print(master_df.head(5))


✅ Final master workbook saved to:
C:\Users\troyr\OneDrive - Self Taught LLC\PythonDev\03-projects\fred-trade-analysis\data\quarterly\master_macro_quarterly_complete_labeled.xlsx

📊 Columns:
['Date', 'Fed Funds Rate', '10-Year Treasury Yield', 'CPI (Urban Consumers)', 'PCE Price Index', 'Personal Savings Rate', 'Unemployment Rate', 'Labor Force Participation', 'Nominal Consumption', 'Real Disposable Income', 'Industrial Production', 'Manufacturing Output', 'Housing Starts', 'Building Permits', 'Capacity Utilization']

📅 First date in dataset: 1972-03-31
        Date  Fed Funds Rate  10-Year Treasury Yield  CPI (Urban Consumers)  \
0 1972-03-31        3.546667                6.033333              41.333333   
1 1972-06-30        4.300000                6.143333              41.600000   
2 1972-09-30        4.743333                6.290000              41.933333   
3 1972-12-31        5.146667                6.373333              42.366667   
4 1973-03-31        6.536667                6.

## 📈 Step 4: Compute Growth Rates, Divergence, Deficits

In [13]:
# Example: Compute Net Exports and GDP Divergence

## 📊 Step 5: Run Correlations, Segment by Regime, Visualize

In [14]:
# Add your correlation analysis and plots here

## 📌 Step 6: Summary Insights and Findings